In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, confusion_matrix

In [3]:
# Parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
DATA_DIR = "./abide2_preprocessed_age_15_or_less"
CSV_FILE = "./abide_2_15_or_less.csv" 

# Load CSV
df = pd.read_csv(CSV_FILE, dtype={'SUB_ID': str})

In [4]:
def load_nii_file(file_path):
    try:
        img = nib.load(file_path).get_fdata()
        if len(img.shape) != 2:
            print(f"Warning: {file_path} has unexpected shape {img.shape}")
            return None
        img = tf.image.resize(img[..., np.newaxis], IMG_SIZE).numpy()
        img = (img - img.min()) / (img.max() - img.min() + 1e-10)
        return img
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return None


In [5]:
# Load images and labels
images, labels = [], []
for idx, row in df.iterrows():
    sub_id = row["SUB_ID"]
    label = row["DX_GROUP"] - 1
    nii_path = os.path.join(DATA_DIR, sub_id, f"{sub_id}_.nii")
    if os.path.exists(nii_path):
        img = load_nii_file(nii_path)
        if img is not None:
            images.append(img)
            labels.append(label)
    else:
        print(f"File not found: {nii_path}")

images = np.array(images)
labels = np.array(labels)
print(f"Total images loaded: {len(images)}, Total labels: {len(labels)}")

if len(images) == 0 or len(labels) == 0:
    raise ValueError("No images or labels loaded.")

File not found: ./abide2_preprocessed_age_15_or_less\28817\28817_.nii
File not found: ./abide2_preprocessed_age_15_or_less\29305\29305_.nii
File not found: ./abide2_preprocessed_age_15_or_less\29327\29327_.nii
Total images loaded: 773, Total labels: 773


In [6]:

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Training set: {len(X_train)}, Validation set: {len(X_val)}")

# Define EfficientNetB0 Model
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
base_model.trainable = False

inputs = Input(shape=(128, 128, 1))
x = Concatenate()([inputs, inputs, inputs])  # Grayscale to RGB
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Training set: 618, Validation set: 155


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ input_layer_1[0]… │
│ (Concatenate)       │ 3)                │            │ input_layer_1[0]… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 4, 4,      │  4,049,571 │ concatenate[0][0] │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    163,968 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [7]:
# Training
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=20,
                    validation_data=(X_val, y_val))

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 78s 877ms/step - accuracy: 0.5224 - loss: 0.7075 - val_accuracy: 0.5226 - val_loss: 0.6925
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 552ms/step - accuracy: 0.5426 - loss: 0.6897 - val_accuracy: 0.4774 - val_loss: 0.7205
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 590ms/step - accuracy: 0.4486 - loss: 0.7387 - val_accuracy: 0.4774 - val_loss: 0.7009
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 529ms/step - accuracy: 0.5148 - loss: 0.7007 - val_accuracy: 0.5226 - val_loss: 0.6922
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 530ms/step - accuracy: 0.5383 - loss: 0.7009 - val_accuracy: 0.4774 - val_loss: 0.6946
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 397ms/step - accuracy: 0.5531 - loss: 0.6856 - val_accuracy: 0.5226 - val_loss: 0.6940
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 522ms/step - accuracy: 0.4939 - loss: 0.7038 - val_accuracy: 0.5226 - val_loss: 0.6932
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 570ms/step - accuracy: 0.5035 - loss: 0.7016 - val_accur

In [10]:
# Inference
NII_FILE = "./abide2_preprocessed_age_15_or_less/30238/30238_.nii"
def load_single_image(file_path):
    img = load_nii_file(file_path)
    return np.expand_dims(img, axis=0)

img = load_single_image(NII_FILE)
prediction = model.predict(img)[0][0]
if prediction >= 0.5:
    print(f"Prediction: ASD (Confidence: {prediction:.4f})")
else:
    print(f"Prediction: TD (Confidence: {1 - prediction:.4f})")

# Evaluation
y_pred_val = model.predict(X_val).flatten()
y_pred_binary = (y_pred_val > 0.5).astype(int)

def evaluate_model(y_true, y_pred, y_prob=None, model_name=""):
    if model_name:
        print(f"--- Performance Metrics for {model_name} ---")

    metrics = {}
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    metrics['Accuracy'] = accuracy
    metrics['F1-Score'] = f1
    metrics['Sensitivity'] = sensitivity
    metrics['Specificity'] = specificity

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Sensitivity (TPR): {sensitivity:.4f}")
    print(f"Specificity (TNR): {specificity:.4f}")

    if y_prob is not None:
        try:
            auc = roc_auc_score(y_true, y_prob)
            metrics['AUC-ROC'] = auc
            print(f"AUC-ROC: {auc:.4f}")
        except ValueError:
            print("Warning: AUC-ROC requires both classes present.")
    return metrics

# Call evaluation
evaluation_metrics = evaluate_model(y_val, y_pred_binary, y_pred_val, model_name="EfficientNetB0 ABIDE")
print("\nValidation Metrics:", evaluation_metrics)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
Prediction: ASD (Confidence: 0.5267)
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 453ms/step
--- Performance Metrics for EfficientNetB0 ABIDE ---
Accuracy: 0.5226
F1-Score: 0.6864
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.0000
AUC-ROC: 0.6180

Validation Metrics: {'Accuracy': 0.5225806451612903, 'F1-Score': 0.6864406779661016, 'Sensitivity': 1.0, 'Specificity': np.float64(0.0), 'AUC-ROC': np.float64(0.6179512846179513)}
